<a href="https://colab.research.google.com/github/JaiKumar1717/FMML-Notebook/blob/main/Lab_4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [275]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [276]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(int)
 #dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


Here is a function for calculating the 1-nearest neighbours

In [277]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [278]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [279]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [280]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [281]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [282]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 45/100)

What is the accuracy of our classifiers on the train dataset?

In [283]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.1641304347826087


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [284]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.329246935201401
Validation accuracy using random classifier is  0.16320052539404553


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [285]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel,   45/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.3243482103402563


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [286]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that these two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Answers

1. Increasing the Percentage of the Validation Set:

  i) More Reliable Estimates: A larger validation set provides a more reliable estimate of the model's performance since it better represents the data distribution.
  ii) Less Training Data: Less data is available for training, which may hinder the model's learning process, potentially reducing its generalizability.

  Reducing the Percentage of the Validation Set:

  i) Less Reliable Estimates: A smaller validation set may not adequately capture the data's variability, leading to less reliable performance metrics.
  ii) More Training Data: More data is available for training, which can improve the model's performance if the training process is effective and avoids overfitting.

2.The size of the training and validation sets directly impacts the model’s ability to generalize:
  i) Large Training Set, Small Validation Set: More data for training can lead to better model learning, but the validation set may not accurately represent the model's true performance, leading to unreliable test set predictions.

  ii) Small Training Set, Large Validation Set: A larger validation set provides better performance estimates, but with less training data, the model might be under-trained, affecting its ability to generalize to the test set.

  Balancing these sizes ensures the model learns effectively while providing reliable validation performance, closely predicting test set accuracy.

3. A commonly used and effective strategy is to reserve about 10-20% of the data for the validation set:

  10% Validation Set: This allows most data for training while providing a good performance estimate.
  20% Validation Set: Provides a more better estimate but slightly less data for training.
  Choosing a percentage within this range generally offers a good balance between training effectiveness and validation accuracy reliability.

For reference: https://1drv.ms/x/s!Aqx4YBZDwpBgpHlULgnDb87QFO2A?e=9IFu7m

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [287]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [274]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


## Answers

1. Yes, averaging the validation accuracy across multiple splits, such as in cross-validation, generally provides more consistent results. This approach mitigates the variance caused by random splits of the data, offering a more stable and reliable estimate of the model’s performance. Each split tests the model on different subsets of the data, and averaging helps smooth out fluctuations that might occur due to specific characteristics of individual subsets.

2. Averaging validation accuracy across multiple splits typically yields a more accurate estimate of test accuracy. This method reduces the impact of any single, potentially unrepresentative split, allowing for a better overall assessment of the model’s generalization capability. It effectively simulates how the model might perform on various unseen datasets, thereby giving a closer approximation of test accuracy.

3. Increasing the number of iterations (e.g., more folds in cross-validation) generally improves the accuracy and reliability of the performance estimate:

  Higher Iterations: Provide a more thorough examination of the model across various splits, leading to a more precise and less biased estimate.
  Diminishing Returns: However, beyond a certain point, additional iterations may yield minimal improvement, especially if the computational cost becomes significant. The improvement tends to stabilize, and the benefit of extra iterations becomes marginal.

4. Yes, increasing the number of iterations can help mitigate the limitations of having small training or validation datasets:

  Small Training Dataset: By iterating through multiple training and validation splits, you maximize the use of available data, allowing the model to learn from various data combinations.
  Small Validation Dataset: More iterations provide diverse validation scenarios, offering a more accurate performance measure even when each individual validation set is small.
In essence, increasing iterations can help extract more robust insights from limited data, enhancing the reliability of the performance estimate despite the small dataset sizes.